In [178]:
from enum import Enum, unique
import toml
from scipy.optimize import fsolve
import numpy as np
from rich.tree import Tree
import pandas as pd

In [179]:
@unique
class ElementTypes(Enum):
    """Element types"""
    SOURCE = 1
    LOAD = 2
    LOSS = 3
    CONVERTER = 4
    LINREG = 5

In [180]:
ElementTypes.SOURCE.name, ElementTypes.SOURCE.value

('SOURCE', 1)

In [307]:
IMAX_DEFAULT = 1.0e6

class ElementMeta(type):
    """An element metaclass that will be used for element class creation.
    """
    def __instancecheck__(cls, instance):
        return cls.__subclasscheck__(type(instance))

    def __subclasscheck__(cls, subclass):
        return (hasattr(subclass, 'solv_inp_curr') and 
                callable(subclass.solv_inp_curr) and 
                hasattr(subclass, 'solv_outp_volt') and 
                callable(subclass.solv_outp_volt) and
                hasattr(subclass, 'element_type') and
                hasattr(subclass, 'child_types'))

class ElementInterface(metaclass=ElementMeta):
    """This interface is used for concrete element classes to inherit from.
    There is no need to define the ElementMeta methods of any class
    as they are implicitly made available via .__subclasscheck__().
    """
    pass
    
    # solver functions
    #def solv_inp_curr(self, vi, vo, io):
    #    return 0.0
    
    #def solv_outp_volt(self, vi, ii, io):
    #    return 0.0 
    

class Source:
    """The Source element must the root of a system. 
    A system can only have one source.
    
    Attributes
    ----------
    element_type : ElementTypes.SOURCE (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.SOURCE
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
           
    def __init__(self, name, *, vo, rs=0.0, imax=IMAX_DEFAULT):
        """Set source name, voltage, internal resistance and max current"""
        self.name = name
        if not (vo > 0):
            raise ValueError('Source voltage must be > 0')
        self.vo = float(vo)
        #if not (voltage > 0):
        #    raise ValueError('Source voltage must be > 0')
        self.imax = imax
        self.rs = rs
        
    @classmethod
    def from_file(cls, name, *, fname=""):
        """Configure source from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        if not 'voltage' in config:
            raise KeyError('Config file is missing entry for \'voltage\'')
        v = config['voltage']
        if 'max_current' in config:    
            mc = config['max_current']
        else:
            mc = -1
        
        return cls(name, voltage=v, max_current=mc)
     
    def get_inp_current(self):
        return 0.0
    
    def get_outp_voltage(self):
        return self.vo
    
    #def set_outp_voltage(self, voltage):
    #    self.vo = float(voltage)
    
    def solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io
        
    def solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.vo - self.rs * io
    
    def solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        pwr = vo * io
        loss = self.rs * io * io
        return pwr, loss, (pwr - loss) / pwr
        
        
class Load:
    """The Load element .
    
    Attributes
    ----------
    element_type : ElementTypes.LOAD (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LOAD
    
    @property
    def child_types(self):
        """The Load element cannot have childs"""
        return None
    
    def __init__(self, name, *, pwr):
        self.name = name
        if not (pwr >= 0):
            raise ValueError('Load input power must be > 0')
        self.pwr = float(pwr)
        
    def get_inp_current(self):
        return 0.0 # TODO if current load -> return load     
    
    def get_outp_voltage(self):
        return 0.0
        
    def solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        if vi > 0:
            return self.pwr / vi
        return 0.0
    
    def solv_outp_volt(self, vi, ii, io):
        """Load output voltage is always 0"""
        return 0.0
    
    def solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        return vi * ii, 0.0, 100.0
     
        
class Loss:
    """The Loss element 
    
    Attributes
    ----------
    element_type : ElementTypes.LOSS (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LOSS
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name, *, res):
        self.name = name
        self.r = float(res)
        
    def get_inp_current(self):
        return 0.0
        
    def get_outp_voltage(self):
        return 0.0
        
    def solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io # TODO: iq
    
    def solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return vi - self.r * io
    
    def solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = self.r * ii * ii
        pwr = vi * ii
        return 0.0, loss, (pwr - loss) / pwr
        
class Converter:  
    """The Converter element 
    
    Attributes
    ----------
    element_type : ElementTypes.LOSS (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.CONVERTER
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name, *, vo, eff, iq=0.0, imax=IMAX_DEFAULT): # add Iq
        self.name = name
        if not (float(vo) > 0.0):
            raise ValueError('Output voltage must be > 0.0')
        self.vo = float(vo)
        if not (float(eff) > 0.0):
            raise ValueError('Efficiency must be > 0.0')
        if not (float(eff) < 1.0):
            raise ValueError('Efficiency must be < 1.0')
        self.eff = float(eff)
        self.imax = float(imax)
        self.iq = float(iq)

    def get_inp_current(self):
        return self.iq
        
    def get_outp_voltage(self):
        return self.vo
           
    def solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        ve = self.eff*vi
        if ve > 0.0:
            return self.iq + vo*io/ve
        return 0.0
    
    def solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.vo
    
    def solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = self.iq * vi + (ii - self.iq) * vi * (1. - self.eff)
        pwr = vi * ii
        return 0.0, loss, (pwr - loss) / pwr
    
class LinReg:  
    """The Linear regulator element 
    
    Attributes
    ----------
    element_type : ElementTypes.LINREG (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LINREG
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name, *, vo, vdrop, iq=0.0, imax=1.0e6):
        self.name = name
        if not (float(vo) > 0.0):
            raise ValueError('Output voltage must be > 0.0')
        self.vo = float(vo)
        if not (float(vdrop) < vo):
            raise ValueError('Voltage drop must be < vo')
        self.vdrop = float(vdrop)
        self.iq = float(iq)
        self.imax = float(imax)
        
    def get_inp_current(self):
        return self.iq
        
    def get_outp_voltage(self):
        return self.vo
    
    def solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io + self.iq
    
    def solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.vo
    
    def solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = (vi - vo) * io + vi * self.iq
        pwr = vi * ii
        return 0.0, loss, (pwr - loss) / pwr

In [308]:
(issubclass(Load, ElementInterface), issubclass(Source, ElementInterface), 
    issubclass(Loss, ElementInterface), issubclass(Converter, ElementInterface), issubclass(LinReg, ElementInterface))

(True, True, True, True, True)

In [309]:
s = Source('test', vo=12)
hasattr(s, 'element_type'), s.child_types

(True,
 [<ElementTypes.LOAD: 2>,
  <ElementTypes.LOSS: 3>,
  <ElementTypes.CONVERTER: 4>,
  <ElementTypes.LINREG: 5>])

In [310]:
import rustworkx as rx
import numpy as np
from scipy.optimize import fsolve, least_squares
from rich.tree import Tree

class System:
    """System to be analyzed.
    """
    
    def __init__(self, name, source):
        self.g = None
        if type(name) != str:
            raise ValueError("Error: Name \"{}\" of system must be a string!".format(name))
        elif not isinstance(source, Source):
            raise ValueError("Error: First element of system must be a source!")
        else:
            self.g = rx.PyDAG(check_cycle=True, multigraph=False, attrs={})
            pidx = self.g.add_node(source)
            self.g.attrs[source.name] = pidx
            
    def __get_index(self, name):
        """Get node index from element name"""
        for n in [0] + list(rx.descendants(self.g, 0)):
            if self.g.get_node_data(n).name == name:
                return n
        return -1
    
    def __chk_parent(self, parent):
        """Check if parent exists"""
        if type(parent) != str:
            raise ValueError("Error: Parent name must be a string!")
        # check if parent exists
        if not parent in self.g.attrs.keys():
            raise ValueError("Error: Parent name \"{}\" not found!".format(parent))
            return False
        
        return True
    
    # check if element name is valid
    def __chk_name(self, name):
        """Check if element name is valid"""
        # check name type
        if type(name) != str:
            raise ValueError("Error: Element name must be a string!")
            return False
        # check if exists exists
        pidx = self.__get_index(name)
        if name in self.g.attrs.keys():
            raise ValueError("Error: Element name \"{}\" is already used!".format(name))
            return False
        
        return True
                  
    def __get_childs(self, rev=True):
        """Get dict of parent/childs"""
        if rev == True:
            childs = list(reversed(rx.bfs_successors(self.g, 0)))
        else:
            childs = list(rx.bfs_successors(self.g, 0))
        cdict = {}
        for c in childs:
            cs = []
            for l in c[1]:
                cs += [self.g.attrs[l.name]]
            cdict[self.g.attrs[c[0].name]] = cs
        return cdict
    
    def __get_nodes(self):
        """Get list of nodes in system"""
        nodes = []
        for n in self.g.nodes():
            nodes += [self.g.attrs[n.name]]
        return sorted(nodes)
    
    def __get_parents(self):
        """Get list of parent of each child"""
        nodes = self.__get_nodes()
        ps = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if n != 0:
                pname = [i for i in self.g.attrs if self.g.attrs[i]==n]
                ps[n] = case1.g.attrs[self.g.predecessors(self.g.attrs[pname[0]])[0].name]
        return list(ps)
    
    def __get_edges(self):
        """Get list of element connections (edges)"""
        return list(reversed(rx.dfs_edges(self.g, 0)))
    
    def __get_leaves(self):
        """Get list of leaf nodes"""
        nodes = self.__get_nodes()
        ls = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if self.g.out_degree(n) == 0:
                ls[n] = 1
        return list(ls)
        
    def __sys_vars(self):
        """Get system variable lists"""
        vn = max(self.__get_nodes())+1 # highest node index + 1
        v = list(np.zeros(vn)) # voltages
        i = list(np.zeros(vn)) # currents
        return v, i 
      
    def __sys_init2(self):
        """Create vector of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self.g[n].get_outp_voltage()
            i[n] = self.g[n].get_inp_current()
        return v+i
    
    def __sys_function2(self, x):
        """System function to be solved."""
        
        # map x to v/i variables
        vi, ii = self.__sys_vars()
        for n in range(len(self.__nodes)):
            vi[self.__nodes[n]] = x[n]
        for e in range(len(self.__edges)):
            ii[self.__edges[e][1]] = x[len(self.__nodes)+e]
        ii[0] = x[-1]
    
        vo, io = self.__sys_vars()
        # update output voltages (per node)
        for n in self.__nodes:
            if self.__leaves[n] == 1: # leaf
                if n == 0: # root             
                    vo[n] = vi[n] - self.g[n].solv_outp_volt(.0, .0, .0)
                else:
                    p = self.__parents[n]
                    vo[n] = vi[n] - self.g[n].solv_outp_volt(vi[p], ii[p], .0)
            else:
                # add currents into childs
                isum = 0
                for c in self.__childs[n]: 
                    isum += ii[c]
                if n == 0: # root
                    vo[n] = vi[n] - self.g[n].solv_outp_volt(.0, .0, isum)
                else:
                    p = self.__parents[n]
                    vo[n] = vi[n] - self.g[n].solv_outp_volt(vi[p], ii[p], isum)
                
        # update input currents (per edge)
        for e in self.__edges:
            if self.__leaves[e[1]] == 1: # leaf
                io[e[1]] = ii[e[1]] - self.g[e[1]].solv_inp_curr(vi[e[0]], .0, .0)
            else:
                c = self.__childs[e[1]]
                io[e[1]] = ii[e[1]] - self.g[e[1]].solv_inp_curr(vi[e[0]], vi[e[1]], ii[c[0]])
        # add currents into childs from root
        isum = 0
        for c in self.__childs[0]:
            isum += ii[c]
        io[0] = ii[0] - isum
            
        # map f(v/i) to f(x)
        y = np.zeros(len(x))
        for n in range(len(self.__nodes)):
            y[n] = vo[self.__nodes[n]]
        for e in range(len(self.__edges)):
            y[len(self.__nodes)+e] = io[self.__edges[e][1]] 
        y[-1] = io[0] 
            
        #print(y)
        return y
    
    def __make_rtree(self, adj, node):
        """Create Rich tree"""
        tree = Tree(node)
        for child in adj.get(node, []):
            tree.add(self.__make_rtree(adj, child))
        return tree
    
    def add_element(self, parent, *, element):
        """Add element to system"""
        if not self.__chk_parent(parent):
            return
        if not self.__chk_name(element.name):
            return
        
        pidx = self.__get_index(parent)
        cidx = self.g.add_child(pidx, element, None)
        self.g.attrs[element.name] = cidx
       
    def tree(self, name=''):
        """Print tree structure, starting from node name"""
        if not name == '':
            if not name in self.g.attrs.keys():
                raise ValueError("Error: Element name is not valid!")
            root = name
        else:
            root = self.g[0].name
            
        adj = rx.bfs_successors(self.g, self.g.attrs[root])
        ndict = {}
        for i in adj:
            c = []
            for j in i[1]:
                c += [j.name]
            ndict[i[0].name] = c
        return self.__make_rtree(ndict, root)
    
    def solve2(self):
        """Analyze system"""
        # create lists of element realtionships for use by __sys_function()
        self.__nodes = self.__get_nodes()
        self.__edges = self.__get_edges()
        self.__childs = self.__get_childs()
        self.__parents = self.__get_parents()
        self.__leaves = self.__get_leaves()
        # solve system function
        res = fsolve(self.__sys_function2, self.__sys_init2(), full_output = True)
        #res = least_squares(self.__sys_function2, self.__sys_init())
        return res
    
    def __sys_init(self):
        """Create vectors of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self.g[n].get_outp_voltage()
            i[n] = self.g[n].get_inp_current()
        return v, i
    
    def __fwd_prop(self, v, i):
        """Forward propagation of voltages"""
        vo, _ = self.__sys_vars()
        # update output voltages (per node)
        for n in self.__nodes:
            if self.__leaves[n] == 1: # leaf
                if n == 0: # root             
                    vo[n] = self.g[n].solv_outp_volt(.0, .0, .0)
                    #print('leaf, root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n].solv_outp_volt(v[p], i[p], .0)
                    #print('leaf: ', vo[n], n)
            else:
                # add currents into childs
                isum = 0
                for c in self.__childs_f[n]: 
                    isum += i[c]
                if n == 0: # root
                    vo[n] = self.g[n].solv_outp_volt(.0, .0, isum)
                    #print('root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n].solv_outp_volt(v[p], i[p], isum)
                    #print('element:' , vo[n], n)
        
        return vo

    def __back_prop(self, v, i):
        """Backward propagation of currents"""
        _, io = self.__sys_vars()
        # update input currents (per edge)
        for e in self.__edges:
            if self.__leaves[e[1]] == 1: # leaf
                io[e[1]] = self.g[e[1]].solv_inp_curr(v[e[0]], .0, .0)
            else:
                c = self.__childs_b[e[1]]
                io[e[1]] = self.g[e[1]].solv_inp_curr(v[e[0]], v[e[1]], i[c[0]])
        # add currents into childs from root
        for c in self.__childs_b[0]:
            io[0] += i[c]
        
        return io
    
    def solve(self, *, vtol=1e-3, itol=1e-6, maxiter=1000):
        """Analyze system"""
        # create lists of element realtionships for use by __sys_function()
        self.__nodes = self.__get_nodes()
        self.__edges = self.__get_edges()
        self.__childs_f = self.__get_childs(rev = False)
        self.__childs_b = self.__get_childs(rev = True)
        self.__parents = self.__get_parents()
        self.__leaves = self.__get_leaves()
        # initial condition
        v, i = self.__sys_init()
        # solve system function
        iters = 1
        while iters <= maxiter:
            vi = self.__fwd_prop(v, i)
            #print(vi)
            ii = self.__back_prop(vi, i)
            #print(ii)
            iters += 1
            
            if np.allclose(np.array(v), np.array(vi), rtol=vtol) and np.allclose(np.array(i), np.array(ii), rtol=itol):
                #print("tol reached")
                break;
            v, i = vi, ii
            
        # create PD frame with result
        res = {}
        names, parent, pwr, loss, eff = [], [], [], [], []
        for n in self.__nodes: # [vi, vo, ii, io]
            names += [self.g[n].name]
            if n == 0: # root 
                p, l, e = self.g[n].solv_pwr_loss(v[n], v[n], i[n], i[n])
                #p = 0.0
                parent += [None]
                #print(n, v[n], v[n], i[n], i[n], p, l)
            elif self.__leaves[n] == 1: # leaf
                p, l, e = self.g[n].solv_pwr_loss(v[self.__parents[n]], v[n], i[n], 0.0)
                parent += [self.g[self.__parents[n]].name]
                #print(n, v[n], 0.0, i[n], 0.0, p, l)
            else:
                isum = 0.0
                for c in self.__childs_f[n]: 
                    isum += i[c]
                p, l, e = self.g[n].solv_pwr_loss(v[self.__parents[n]], v[n], i[n], isum)
                parent += [self.g[self.__parents[n]].name]
                #print(n, v[self.__parents[n]], v[n], i[n], isum, p, l)
                
            pwr += [p]
            loss += [l]
            eff += [e]
            
        
        res['Element'] = names
        res['Parent'] = parent
        res['Vout (V)'] = v
        res['Iin (A)'] = i
        res['Power (W)'] = pwr #[res['Power (W)'].sum() - res['Power (W)'][0]]
        res['Loss (W)'] = loss #[res['Loss (W)'].sum()]
        res['Efficiency (%)'] = eff
        df = pd.DataFrame(res)
        #df['Total']['Power (W)'] = df['Power (W)'].sum() - df['Power (W)'][0]
        #df['Total']['Loss (W)'] = df['Loss (W)'].sum()
        #tpwr = df['Power (W)'].sum() - df['Power (W)'][0]
        tpwr = v[0] * i[0]
        tloss = df['Loss (W)'].sum()
        df.loc[len(df)] = ['System total', None, v[0], i[0], tpwr, tloss, (tpwr - tloss)/tpwr]
         
        return df

![case1](case_1.svg)

In [313]:
case1 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
case1.add_element(parent='3V coin', element=Loss('Resistor', res=33.0))
case1.add_element('Resistor', element=Converter('1.8V buck', vo=1.8, eff=0.87, iq=12e-6))
case1.add_element('1.8V buck', element=Load('MCU', pwr=27e-3))
case1.add_element('3V coin', element=Converter('5V boost', vo=5, eff=0.91, iq=42e-6))
case1.add_element('5V boost', element=Load('Sensor', pwr=15e-3))
case1.add_element('5V boost', element=LinReg('LDO 2.5V', vo=2.5, vdrop=0.27, iq=150e-6))
case1.add_element('LDO 2.5V', element=Load('ADC', pwr=15e-3))
#case1.g.attrs
case1.tree()

3V coin
├── 5V boost
│   ├── LDO 2.5V
│   │   └── ADC
│   └── Sensor
└── Resistor
    └── 1.8V buck
        └── MCU

In [314]:
df = case1.solve()
df

,Element,Parent,Vout (V),Iin (A),Power (W),Loss (W),Efficiency (%)
0,3V coin,None,2.999698,0.023226,0.069671,0.000007,0.999899
1,Resistor,3V coin,2.606365,0.011919,0.000000,0.004688,0.868876
2,1.8V buck,Resistor,1.800000,0.011919,0.000000,0.004066,0.869124
3,MCU,1.8V buck,0.000000,0.015000,0.027000,0.000000,100.000000
4,5V boost,3V coin,5.000000,0.011307,0.000000,0.003167,0.906620
5,Sensor,5V boost,0.000000,0.003000,0.015000,0.000000,100.000000
6,LDO 2.5V,5V boost,2.500000,0.006150,0.000000,0.015750,0.487805
7,ADC,LDO 2.5V,0.000000,0.006000,0.015000,0.000000,100.000000
8,System total,None,2.999698,0.023226,0.069671,0.027678,0.602731


In [194]:
df['Power (W)'].sum() - df['Power (W)'][0]

0.05699999999999998

In [95]:
#case1._System__childs_f
#su = rx.bfs_successors(case1.g, 0)
su = rx.topological_sort(case1.g)
for i in su:
    print(i, case1.g[i].name)

0 3V coin
4 5V boost
6 LDO 2.5V
7 ADC
5 Sensor
1 Resistor
2 1.8V buck
3 MCU


In [52]:
np.allclose(np.array(v), np.array(v)*1.001, rtol=1e-3)

True

In [36]:
res = case1.solve2()
res[0]

array([ 2.99862512e+00, -8.19390188e-12,  1.80000000e+00,  0.00000000e+00,
        4.99999999e+00,  0.00000000e+00,  2.50000000e+00,  0.00000000e+00,
        6.00000000e-03,  6.14783077e-03,  3.00000000e-03,  1.13291237e-02,
        1.50000000e-02,  9.07841867e-02,  9.08674277e-02,  1.02173211e-01])

In [62]:
res = case1.solve()
res[0]

array([3.00000000e+00, 2.60718609e+00, 1.80000000e+00, 0.00000000e+00,
       5.00000000e+00, 0.00000000e+00, 2.50000000e+00, 0.00000000e+00,
       6.00000000e-03, 6.15000000e-03, 3.00000000e-03, 1.13057363e-02,
       1.50000000e-02, 1.19034518e-02, 1.19034518e-02, 2.32091881e-02])

In [78]:
%timeit case1.solve()

10.8 ms ± 51.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%timeit case1.solve()

4.18 ms ± 7.27 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [151]:
rx.bfs_successors(case1.g, self.g.attrs[])

'3V coin'

In [22]:
# get system variables
def sys_vars(sys):
    vn = max(sys.get_nodes())+1 # highest node index + 1
    v = list(np.zeros(vn)) # voltages
    i = list(np.zeros(vn)) # currents
    return v, i

# get initialization vector
def sys_init(sys):
    v, i = sys_vars(sys)
    for n in sys.get_nodes():
        #v[n] = sys.g[n].get_outp_voltage()
        v[n] = np.random.uniform(0.5, 1.5)
        i[n] = np.random.uniform(0.001, 0.2)
        
    return v+i

def sys_function(x, nodes, edges, childs, parents, leaves, sys):
    # map x to v/i variables
    vi, ii = sys_vars(sys)
    for n in range(len(nodes)):
        vi[nodes[n]] = x[n]
    for e in range(len(edges)):
        ii[edges[e][1]] = x[len(nodes)+e]
    ii[0] = x[-1]
    
    vo, io = sys_vars(sys)
    # update output voltages (per node)
    for n in nodes:
        if leaves[n] == 1: # leaf
            if n == 0: # root             
                vo[n] = vi[n] - sys.g[n].solv_outp_volt(.0, .0, .0)
            else:
                p = parents[n]
                vo[n] = vi[n] - sys.g[n].solv_outp_volt(vi[p], ii[p], .0)
        else:
            # add currents into childs
            isum = 0
            for c in childs[n]: 
                isum += ii[c]
            if n == 0: # root
                vo[n] = vi[n] - sys.g[n].solv_outp_volt(.0, .0, isum)
            else:
                p = parents[n]
                vo[n] = vi[n] - sys.g[n].solv_outp_volt(vi[p], ii[p], isum)
                
    # update input currents (per edge)
    for e in edges:
        if leaves[e[1]] == 1: # leaf
            io[e[1]] = ii[e[1]] - sys.g[e[1]].solv_inp_curr(vi[e[0]], .0, .0)
        else:
            c = childs[e[1]]
            io[e[1]] = ii[e[1]] - sys.g[e[1]].solv_inp_curr(vi[e[0]], vi[e[1]], ii[c[0]])
    # add currents into childs from root
    isum = 0
    for c in childs[0]:
        isum += ii[c]
    io[0] = ii[0] - isum
            
    # map f(v/i) to f(x)
    y = np.zeros(len(x))
    for n in range(len(nodes)):
        y[n] = vo[nodes[n]]
    for e in range(len(edges)):
        y[len(nodes)+e] = io[edges[e][1]] 
    y[-1] = io[0] 
            
    #print(y)
    return y

In [23]:
sys_function(sys_init(case1), case1.get_nodes(), case1.get_edges(), case1.get_childs(), 
             case1.get_parents(), case1.get_leaves(), case1)

#sys_init(case1)

array([-1.50314693, 16.35832126, -0.6954055 ,  0.68606533, -3.68370378,
        1.21017742,  1.20789251, -0.44208121,  1.45990493, -1.47881996,
        0.58696221, -0.52109743])

In [24]:
res = fsolve(sys_function, 
       sys_init(case1), 
       args=(case1.get_nodes(), case1.get_edges(), case1.get_childs(), case1.get_parents(), case1.get_leaves(), case1),
       full_output = True)
res[0], res[1]['nfev']

(array([ 3.00000000e+00,  2.60718656e+00,  1.80000000e+00, -2.43026589e-41,
         5.00000000e+00,  9.47185335e-42,  3.00000000e-03,  5.49450549e-03,
         1.50000000e-02,  1.19034377e-02,  1.19034377e-02,  1.73979432e-02]),
 38)

array([3.00000000e+00, 2.60718656e+00, 1.80000000e+00, 0.00000000e+00,
       5.00000000e+00, 0.00000000e+00, 3.00000000e-03, 5.49450549e-03,
       1.50000000e-02, 1.19034377e-02, 1.19034377e-02, 1.73979432e-02])

In [75]:
case1.__methods__

AttributeError: 'system' object has no attribute '__methods__'

In [13]:
n = rx.bfs_successors(case1.g, 0)
for e in reversed(n):
    print(e[0].name, e[0].element_type.name, [x.name for x in e[1]])

1.8V buck CONVERTER ['MCU']
Resistor LOSS ['1.8V buck']
5V boost CONVERTER ['Sensor']
3V coin SOURCE ['5V boost', 'Resistor']


In [75]:
e[0].get_inp_voltage()

3.0

In [69]:
# end nodes:
[x for x in case1.g.nodes() if case1.g.out_degree(case1.g.attrs[x.name])==0 and case1.g.in_degree(case1.g.attrs[x.name])==1]

[<__main__.Load at 0x1a08ce39d30>, <__main__.Load at 0x1a08cdfff10>]

Solve for:
* input current
* output voltage

In [33]:
for e in case1.g.nodes():
    print(e.get_inp_voltage())

3.0
0.0
0.0
0.0
0.0


(0.7299270072992701, 10.0)

In [ ]:
e2 = Load("test", current=7)
e2.i

In [34]:
a=list(ElementTypes)
a.remove(ElementTypes.SOURCE)
a

[<ElementTypes.LOAD: 2>, <ElementTypes.LOSS: 3>, <ElementTypes.CONVERTER: 4>]

In [58]:
s = Source("main", voltage=230)
s.__dict__

{'name': 'main', 'vo': 230, 'max_io': -1}

In [8]:
with open('source1.toml', 'w') as f:
    toml.dump({'voltage': 60.2, 'max_current': 0.72, 'comment': "test string"}, f)

In [9]:
s2 = Source.from_file("test2", fname="source1.toml")
s2.__dict__

{'name': 'test2', 'vo': 60.2, 'max_io': 0.72}

In [59]:
s.get_inp_voltage()

230

In [52]:
def make_rtree(adj, node):
    tree = Tree(node)
    for child in adj.get(node, []):
        tree.add(make_rtree(adj, child))
    return tree

adj = rx.bfs_successors(case1.g, 0)
ndict = {}
for i in adj:
    c = []
    for j in i[1]:
        c += [j.name]
    ndict[i[0].name] = c
make_rtree(ndict, "3V coin")

3V coin
├── 5V boost
│   └── Sensor
└── Resistor
    └── 1.8V buck
        └── MCU

In [51]:
for i in rx.bfs_successors(case1.g, 0):
    print(i[1].name)

AttributeError: 'list' object has no attribute 'name'

In [60]:
s

3V coin
├── 5V boost
│   └── Sensor
└── Resistor
    └── 1.8V buck
        └── MCU

In [61]:
type(s)

rich.tree.Tree